#This is my take on the task for Day 5 

I use geminis free cloud tier API keys and work with the OpenAI SDK to make use of it 
-This can be used as an alternative to paid API keys or if you dont want to use OpenAI or cant Afford API keys :)
-Do not forget to activate python kernel

In [ ]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [ ]:
secret = os.getenv('GEMINI_API_KEY')

model = 'gemini-2.5-pro'

def check_key():

    if secret and secret.startswith('AIz') and len(secret) > 10:

        print("Secret located")
    
    else:

        print("Secret invalid / not found")

client = OpenAI(
    api_key= secret,
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
)

located_secret = check_key()

located_secret


In [ ]:
links = fetch_website_links("https://www.six6marketing.co.za/")
links

In [ ]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
print(get_links_user_prompt("https://www.six6marketing.co.za/"))

In [ ]:
def select_relevant_links(url):

    print(f"Selecting relevant links for {url} by calling {model}")
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://www.six6marketing.co.za/")

#Second Half

Make the Brochure

In [ ]:
#testing function in this block 

contents = fetch_website_contents("https://www.six6marketing.co.za/")
print(contents)

In [ ]:
#This function is going to generate the contents for each of the relevant links

def fetch_page_and_all_relevant_links(url):

    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"##Landing Page:  \n {contents} \n \n #Relevant Link {relevant_links} \n "

    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result


In [ ]:
print(fetch_page_and_all_relevant_links("https://www.six6marketing.co.za/"))

In [ ]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [ ]:
def brochure_user_prompt(company_name, url):

    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
print(brochure_user_prompt("Six6 Marketing", "https://www.six6marketing.co.za/"))

In [ ]:
def generate_sales_brochure(company_name, url):

    response = client.chat.completions.create(
        model= model,
        messages=[
            {"role": "system", "content" : brochure_system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name,url)}
        ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
generate_sales_brochure("Six6","https://www.six6marketing.co.za/")

In [ ]:
def stream_brochure(company_name, url):
    stream = client.chat.completions.create(
        model= model,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("Six6","https://www.six6marketing.co.za/")